# Projeto ADEL (Análise Dados Estadia em Limeira) - Elegir bairros distantes
Neste notebook, determinarei as linhas de ônibus que passam pelos "bairros distantes" até a faculdade.

In [1]:
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

## Importando os dados

In [4]:
from pathlib import Path
import glob
import shutil

def moveFilesByExtension(fileExtension, origin, to):
  arq_linhas = []
  arq_linhas += glob.glob(fileExtension)

  # Mover cada arquivo para a pasta desejada
  for arquivo in arq_linhas:
    source = Path(origin + arquivo)
    destination = Path(to + '/' + arquivo)
    shutil.move(source, destination)
  print(f"Foram movidos {len(arq_linhas)} arquivos {fileExtension.replace('*', '')}")

In [5]:
linhas_onibus_files = files.upload()

Saving linha_01.csv to linha_01.csv
Saving linha_02.csv to linha_02.csv
Saving linha_03.csv to linha_03.csv
Saving linha_04.csv to linha_04.csv
Saving linha_06.csv to linha_06.csv
Saving linha_08B.csv to linha_08B.csv
Saving linha_08C.csv to linha_08C.csv
Saving linha_09.csv to linha_09.csv
Saving linha_10.csv to linha_10.csv
Saving linha_11.csv to linha_11.csv
Saving linha_11B.csv to linha_11B.csv
Saving linha_12.csv to linha_12.csv
Saving linha_13.csv to linha_13.csv
Saving linha_15.csv to linha_15.csv
Saving linha_102.csv to linha_102.csv
Saving linha_103.csv to linha_103.csv
Saving linha_104.csv to linha_104.csv
Saving linha_105.csv to linha_105.csv
Saving linha_107.csv to linha_107.csv
Saving linha_114.csv to linha_114.csv
Saving linha_115.csv to linha_115.csv
Saving linha_124.csv to linha_124.csv
Saving linha_400.csv to linha_400.csv
Saving linha_410.csv to linha_410.csv
Saving linha_420.csv to linha_420.csv
Saving linha_430.csv to linha_430.csv
Saving linha_440.csv to linha_440.

In [6]:
# Movendo os dados das linhas para a pasta de linhas
moveFilesByExtension("*.csv", '', 'Data/Linhas-onibus')

Foram movidos 27 arquivos .csv


In [7]:
bairros_distantes_file = files.upload()

Saving bairros_distantes.csv to bairros_distantes.csv


## Carregando os dados

In [8]:
df_bairros = pd.read_csv('Data/bairros_distantes.csv')
print(df_bairros.shape)
df_bairros.head()

(13, 4)


,bairro,lat,lon,dist
0,vila rocha,-22.577952,-47.410891,2.243706
1,vila anita,-22.556810,-47.410568,1.508559
2,vila santa rosalia,-22.576232,-47.406310,2.429568
3,Centro,-22.565492,-47.403798,2.134072
4,Chácara Antonieta,-22.555338,-47.436975,1.502339


In [110]:
# Pegar todos os datasets de linhas de onibus
dir_linhas = []
dir_linhas += glob.glob('Data/Linhas-onibus/*.csv')

# Guardar o label de cada linha
dict_linhas = {}
id_linha = 0

# Dataframe com todas as linhas
df_linhas = pd.DataFrame(data=None, columns=['num_ponto', 'ponto', 'latitude', 'longitude', 'distancia_unicamp', 'id_linha'])

for linha in dir_linhas:
  # Carregar dataset
  df = pd.read_csv('{}'.format(linha))

  # Mapear label
  nome_linha = linha.split('/')[-1].split('.')[0].replace('*', '')
  dict_linhas[id_linha] = nome_linha

  # Colocar a identificação da linha
  df['id_linha'] = id_linha
  id_linha += 1

  # Concatenar dataframes
  df_linhas = pd.concat([df_linhas, df])
  
  # Identificação
  print('*'*40)
  print('{} - {}'.format(nome_linha, df.shape))

****************************************
linha_104 - (53, 6)
****************************************
linha_105 - (39, 6)
****************************************
linha_02 - (45, 6)
****************************************
linha_12 - (38, 6)
****************************************
linha_13 - (54, 6)
****************************************
linha_124 - (51, 6)
****************************************
linha_410 - (8, 6)
****************************************
linha_03 - (43, 6)
****************************************
linha_09 - (9, 6)
****************************************
linha_420 - (18, 6)
****************************************
linha_115 - (27, 6)
****************************************
linha_440 - (7, 6)
****************************************
linha_114 - (58, 6)
****************************************
linha_103 - (73, 6)
****************************************
linha_11 - (32, 6)
****************************************
linha_102 - (45, 6)
*********************************

In [111]:
df_linhas

,num_ponto,ponto,latitude,longitude,distancia_unicamp,id_linha
0,1,Ponto Final B.Ometto,-22.552361,-47.450262,2.882341,0
1,2,"Rua Chile, 142",-22.558130,-47.455177,3.214996,0
2,3,"Avenida Canadá, 2590-2638",-22.549492,-47.447596,2.771835,0
3,4,"Avenida Canadá, 1114-1304",-22.549492,-47.447596,2.771835,0
4,5,"Avenida Canadá, 1306-1558",-22.549492,-47.447596,2.771835,0
...,...,...,...,...,...,...
38,39,"Rua Rosália Schimidt De Camargo Aranha, 369-421",-22.596021,-47.387415,5.346584,26
39,40,"Rua Lourival Berg, 119",-22.596375,-47.387084,5.398407,26
40,41,"Rua José Deon, 243-299",-22.594969,-47.388415,5.191548,26
41,42,"Rua Dário Roland, 927-1017",-22.597896,-47.399240,4.741601,26


In [112]:
df_linhas['id_linha'].value_counts().sort_index()

0     53
1     39
2     45
3     38
4     54
5     51
6      8
7     43
8      9
9     18
10    27
11     7
12    58
13    73
14    32
15    45
16    34
17    21
18    13
19    65
20    36
21    14
22    46
23    49
24    49
25    23
26    43
Name: id_linha, dtype: int64

In [113]:
df_linhas['id_linha'].value_counts().sum()

993

In [114]:
# Identificar o id da linha com o nome
label_linhas = pd.Series(dict_linhas)
label_linhas

0     linha_104
1     linha_105
2      linha_02
3      linha_12
4      linha_13
5     linha_124
6     linha_410
7      linha_03
8      linha_09
9     linha_420
10    linha_115
11    linha_440
12    linha_114
13    linha_103
14     linha_11
15    linha_102
16     linha_04
17     linha_10
18     linha_15
19     linha_06
20     linha_01
21    linha_430
22    linha_08B
23    linha_107
24    linha_11B
25    linha_400
26    linha_08C
dtype: object

In [115]:
# Numero de pontos de onibus que se repetem em mais de uma linha
df_linhas.iloc[:, :-1].duplicated().sum()

31

## Descobrir as linhas que levam até a faculdade

In [118]:
import geopy.distance

def dentro_do_raio(coord1, coord2, raio):
  dist = geopy.distance.vincenty(coord1, coord2).km
  if dist > raio:
    return False
  else:
    return True

def get_id_linha(pontos_perto):
  return df_linhas.iloc[pontos_perto].id_linha.unique()

In [198]:
lista_bairros = list(df_bairros.bairro)
lista_linhas = list(label_linhas.sort_values())

df_bairros_linhas = pd.DataFrame(data=None, columns=lista_bairros, index=lista_linhas)
df_bairros_linhas

,vila anita,vila santa rosalia,Centro,Chácara Antonieta,Jardim Maria Flora,Jardim Mediterrâneo,Jardim Mercedes,Vila Cláudia,Vila Cristovam,Vila Santa Lina
linha_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_08B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_08C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linha_102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
coords_ft_unicamp = (-22.561883227911103, -47.42418079671464)

# Determinar as linhas de cada bairro
for bairro_index in range(df_bairros.shape[0]):
  bairro_coord = df_bairros.iloc[bairro_index, 1:-1]
  bairro_nome = df_bairros.iloc[bairro_index, 0]
  pontos_perto = []

  print('*'*60)
  print(f'Encontrando linhas de ônibus que levam de {bairro_nome} até a faculdade...')

  # Percorrer todos os pontos e descobrir os pertos
  for ponto_index in range(df_linhas.shape[0]):
    ponto_coord = df_linhas.iloc[ponto_index, 2:-1]
    if dentro_do_raio(bairro_coord, ponto_coord, 0.5):
      pontos_perto.append(ponto_index)

  # Descobrir os id's das linhas dos pontos achados
  get_id_linha(pontos_perto)

  id_linhas_encontrados = []

  # Verificar se alguma linha leva até a faculdade
  for i in range(len(get_id_linha(pontos_perto))):
    id_linha = get_id_linha(pontos_perto)[i]
    df_linha = df_linhas.query('id_linha == {}'.format(id_linha))

    # Verificar se algum ponto da linha eh perto da faculdade
    for j in range(len(df_linha)):
      ponto_coord = df_linha.iloc[j, 2:-2] # coordenadas do ponto de onibus
      if dentro_do_raio(coords_ft_unicamp, ponto_coord, 0.4):
        print('-'*60)
        print(f'A linha de id {id_linha} leva do bairro {bairro_nome} até a faculdade')
        id_linhas_encontrados.append(id_linha)
        break

  # Preencher dataframe com os dados de linhas por bairros
  for linha_nome in label_linhas.loc[id_linhas_encontrados].sort_values():
    df_bairros_linhas.loc[linha_nome, bairro_nome] = 'Sim'

  print('*'*60)
  print()

************************************************************
Encontrando linhas de ônibus que levam de vila anita até a faculdade...
------------------------------------------------------------
A linha de id 0 leva do bairro vila anita até a faculdade
------------------------------------------------------------
A linha de id 2 leva do bairro vila anita até a faculdade
------------------------------------------------------------
A linha de id 3 leva do bairro vila anita até a faculdade
------------------------------------------------------------
A linha de id 15 leva do bairro vila anita até a faculdade
************************************************************

************************************************************
Encontrando linhas de ônibus que levam de vila santa rosalia até a faculdade...
------------------------------------------------------------
A linha de id 2 leva do bairro vila santa rosalia até a faculdade
------------------------------------------------------------

In [200]:
df_bairros_linhas = df_bairros_linhas.dropna(how='all', axis=0).dropna(how='all', axis=1).fillna('Não')
df_bairros_linhas.index.name = 'linha'
df_bairros_linhas

,vila anita,vila santa rosalia,Centro,Chácara Antonieta,Jardim Maria Flora,Jardim Mediterrâneo,Jardim Mercedes,Vila Cláudia,Vila Cristovam,Vila Santa Lina
linha,,,,,,,,,,
linha_02,Sim,Sim,Sim,Sim,Não,Não,Sim,Não,Sim,Não
linha_06,Não,Não,Sim,Não,Não,Sim,Não,Não,Não,Não
linha_102,Sim,Não,Sim,Não,Não,Não,Sim,Não,Não,Não
linha_104,Sim,Não,Sim,Sim,Não,Não,Sim,Não,Não,Não
linha_105,Não,Não,Sim,Sim,Não,Não,Não,Não,Não,Não
linha_12,Sim,Não,Sim,Sim,Não,Não,Não,Não,Não,Não
linha_13,Não,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim


In [189]:
bairros_distantes_com_linhas = (
    (df_bairros['bairro'] == 'vila anita') |
    (df_bairros['bairro'] == 'vila santa rosalia') |
    (df_bairros['bairro'] == 'Centro') |
    (df_bairros['bairro'] == 'Chácara Antonieta') |
    (df_bairros['bairro'] == 'Jardim Maria Flora') |
    (df_bairros['bairro'] == 'Jardim Mediterrâneo') |
    (df_bairros['bairro'] == 'Jardim Mercedes') |
    (df_bairros['bairro'] == 'Vila Cláudia') |
    (df_bairros['bairro'] == 'Vila Cristovam') |
    (df_bairros['bairro'] == 'Vila Esteves 	') |
    (df_bairros['bairro'] == 'Vila Santa Lina')
)

df_bairros = df_bairros.loc[bairros_distantes_com_linhas].reset_index(drop=True)
df_bairros

,bairro,lat,lon,dist
0,vila anita,-22.556810,-47.410568,1.508559
1,vila santa rosalia,-22.576232,-47.406310,2.429568
2,Centro,-22.565492,-47.403798,2.134072
3,Chácara Antonieta,-22.555338,-47.436975,1.502339
4,Jardim Maria Flora,-22.583357,-47.400116,3.432218
5,Jardim Mediterrâneo,-22.575642,-47.412714,1.926653
6,Jardim Mercedes,-22.570169,-47.403409,2.325004
7,Vila Cláudia,-22.582487,-47.401728,3.246188
8,Vila Cristovam,-22.574562,-47.404341,2.476853
9,Vila Santa Lina,-22.575128,-47.401097,2.790540


## Exportar dados

In [190]:
df_bairros.to_csv(r'bairros_distantes_com_linhas.csv', index=False)
files.download('bairros_distantes_com_linhas.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [201]:
df_bairros_linhas.to_csv(r'bairros_e_suas_linhas.csv', index=True)
files.download('bairros_e_suas_linhas.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>